In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import shapely

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

from utilities import *
import shared_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
analysis_date = dt.date(2022, 5, 4) ## Wed, May 4

In [3]:
all_hqta = gpd.read_parquet(f'{GCS_FILE_PATH}export/{analysis_date.isoformat()}/ca_hq_transit_areas.parquet')

In [4]:
all_hqta >> head(3)

,calitp_itp_id_primary,agency_name_primary,hqta_type,calitp_itp_id_secondary,agency_name_secondary,geometry
0,284,San Joaquin Regional Transit District,major_stop_bus,284,San Joaquin Regional Transit District,"POLYGON ((-110954.624 -5674.094, -110958.500 -..."
1,284,San Joaquin Regional Transit District,major_stop_bus,284,San Joaquin Regional Transit District,"POLYGON ((-111070.873 -5698.324, -111074.750 -..."
2,284,San Joaquin Regional Transit District,major_stop_bus,284,San Joaquin Regional Transit District,"POLYGON ((-111070.746 -5703.829, -111074.622 -..."


In [5]:
all_hqta = all_hqta.dissolve('calitp_itp_id_primary')

In [6]:
m = map_hqta(all_hqta, mouseover='agency_name_primary', name='HQTA')

In [7]:
green_means_go = gpd.read_parquet(f'{GCS_FILE_PATH}external_sources/green_means_go.parquet')

In [8]:
green_means_go >> head(3)

,FID,Id,GZID,Juris,GZName,Status,Shape__Area,Shape__Length,geometry
0,1,0,GZ06,City of Davis,Downtown Davis,Adopted,6.920288e+06,15464.497947,"POLYGON ((-121.74042 38.55170, -121.74027 38.5..."
1,2,0,GZ07,City of Davis,2nd/5th Street Corridor,Adopted,1.643444e+07,31257.596190,"POLYGON ((-121.73535 38.54955, -121.73545 38.5..."
2,3,0,GZ08,City of Davis,University Research Park,Adopted,4.420622e+06,11439.485450,"POLYGON ((-121.73709 38.53871, -121.73556 38.5..."


In [9]:
itp_green = shared_utils.calitp_color_palette.CALITP_CATEGORY_BOLD_COLORS[3]

In [10]:
geo_data_green = GeoData(geo_dataframe = green_means_go.to_crs('EPSG:4326'),
                       style={'color': 'green', 'fillColor': itp_green,
                                    'opacity':0.2, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Green Means Go Zones')

In [11]:
m.add_layer(geo_data_green)

### A quick map of HQTAs and SACOG Green Means Go Zones:

* Use the layer control under the zoom buttons to toggle each layer
* Mouse over an HQTA to show the transit agency

In [12]:
m

Map(center=[37.77129803905358, -122.20938745120596], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Better Map

In [13]:
popup_dict = {"agency_name_primary": 'Agency Name'}
popup_dict_green = {"GZName": "Green Means Go Zone Name"}

In [26]:
LAYERS_DICT = {
    "HQTA": {"df": all_hqta,
        "plot_col": "agency_name_primary",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: itp_blue,
        'style_function': lambda x: {'fillOpacity': .5, 'fillColor': itp_blue, 'stroke': False}
    },
    "Green Means Go": {"df": green_means_go,
        "plot_col": "GZName",
        "popup_dict": popup_dict_green, 
        "tooltip_dict": popup_dict_green,
        "colorscale": lambda x: itp_green,
        'style_function': lambda x: {'fillOpacity': .5, 'fillColor': itp_green, 'stroke': False}
    },
}

In [27]:
shared_utils.map_utils.make_geojson_layer(LAYERS_DICT['Green Means Go'], 'test')

In [28]:
fig = shared_utils.map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 800, fig_height = 800, 
        zoom=9, 
        centroid = [36.5, -120], 
        title=f"HQTAs and Green Means Go Zones",
    )

In [30]:
fig.save('hqta_green.html')